# MuSiQue baselines

In [ ]:
#|default_exp musique.baseline

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export

import json
from typing import Callable

import pandas as pd
from tqdm.auto import tqdm

from bellek.jerx.reward.llm import QuestionAnsweringResult
from bellek.musique.eval import calculate_metrics, compare_answers

tqdm.pandas()

In [ ]:
#|export

def make_docs(example):
    ps = example["paragraphs"]
    for p in ps:
        idx = p["idx"]
        title = p["title"]
        body = p["paragraph_text"]
        is_supporting = p["is_supporting"]
        text = f"# {title}\n{body}"
        yield dict(
            text=text,
            is_supporting=is_supporting,
            parent_id=example["id"],
            id=idx,
        )

In [ ]:
#|export

class BaselineSingleHop:
    def __init__(self, qa_func, retrieval_func):
        self.qa_func = qa_func
        self.retrieval_func = retrieval_func

    def _call(self, example) -> QuestionAnsweringResult:
        docs = list(make_docs(example))
        retrieved_docs = self.retrieval_func(docs, example['question'])
        context = "\n\n".join([doc["text"] for doc in retrieved_docs])
        return self.qa_func(context=context, question=example['question'])

    def __call__(self, example, ignore_errors: bool = False) -> QuestionAnsweringResult:
        try:
            output = self._call(example)
        except Exception as exc:
            if ignore_errors:
                id = example['id']
                print(f"Failed to answer the question {id}\n{exc}")
                output = QuestionAnsweringResult(reasoning="", answer="N/A", raw_output=str(exc))
            else:
                raise
        return output

In [ ]:
#|export

class BaselineMultiHop:
    def __init__(self, qa_func, retrieval_func):
        self.qa_func = qa_func
        self.retrieval_func = retrieval_func

    def _call(self, example) -> QuestionAnsweringResult:
        docs = list(make_docs(example))
        
        # First question
        question1 = example["question_decomposition"][0]["question"]
        query1 = question1
        docs1 = self.retrieval_func(docs, query1)
        context1 = "\n".join(doc['text'] for doc in docs1)
        result1 = self.qa_func(context=context1, question=question1)
        hop1 = {
            "question": question1,
            "query" : query1,
            "context": context1,
            "answer": result1.answer,
            "reasoning": result1.reasoning,
        }

        # Second question
        if result1.answer == "N/A":
            return {
                "answer": "N/A",
                "reasoning": result1.reasoning,
                "hops": [hop1],
            }

        question2 = example["question_decomposition"][1]["question"]
        question2 = question2.replace("#1", result1.answer)
        query2 = question2
        docs2 = self.retrieval_func(docs, query2)
        context2 = "\n".join(doc['text'] for doc in docs2)
        result2 = self.qa_func(context=context2, question=question2)
        hop2 = {
            "question": question2,
            "query": query2,
            "context": context2,
            "answer": result2.answer,
            "reasoning": result2.reasoning,
        }
        return QuestionAnsweringResult(answer=result2.answer, reasoning=result2.reasoning, raw_output=json.dumps([hop1, hop2]))

    def __call__(self, example, ignore_errors: bool = False) -> QuestionAnsweringResult:
        try:
            output = self._call(example)
        except Exception as exc:
            if ignore_errors:
                id = example['id']
                print(f"Failed to answer the question {id}\n{exc}")
                output = QuestionAnsweringResult(reasoning="", answer="N/A", raw_output=str(exc))
            else:
                raise
        return output

In [ ]:
#|export

def benchmark(
    dataf: pd.DataFrame,
    pipeline: Callable,
    ignore_errors: bool = False,
) -> tuple[pd.DataFrame, dict]:

    def process(example):
        output = pipeline(example, ignore_errors=ignore_errors)
        example["predicted_answer"] = output.answer
        example["raw_llm_output"] = output
        return example

    dataf = dataf.progress_apply(process, axis=1)
    dataf = compare_answers(dataf)
    scores = calculate_metrics(dataf)
    scores["fuzzy_match"] = dataf["fuzzy_match"].mean()
    return dataf, scores

In [ ]:
df = pd.read_json('../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
print(len(df))
df.head()

100


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__131818_161450,"[{'idx': 0, 'title': 'Maria Carrillo High Scho...",Where is the Voshmgir District located?,"[{'id': 131818, 'question': 'Which state is Vo...",in the north-east of the country south of the ...,"[Caspian Sea, in the north-east of the country...",True,"[Caspian Sea, in the north-east of the country..."
1,2hop__444265_82341,"[{'idx': 0, 'title': 'Ocala, Florida', 'paragr...",In what part of Florida is Tom Denney's birthp...,"[{'id': 444265, 'question': 'Tom Denney >> pla...",in Northern Florida,"[in Northern Florida, Northern Florida]",True,"[in Northern Florida, Northern Florida]"
2,2hop__711946_269414,"[{'idx': 0, 'title': 'Wild Thing (Tone Lōc son...",What record label is the performer who release...,"[{'id': 711946, 'question': 'All Your Faded Th...",Kill Rock Stars,[Kill Rock Stars],True,[Kill Rock Stars]
3,2hop__311931_417706,"[{'idx': 0, 'title': 'The Main Attraction (alb...",What record label does the performer of Emotio...,"[{'id': 311931, 'question': 'Emotional Rain >>...",Attic Records,"[Attic, Attic Records]",True,"[Attic, Attic Records]"
4,2hop__809785_606637,"[{'idx': 0, 'title': 'The Main Attraction (alb...",What record label does the performer of Advent...,"[{'id': 809785, 'question': 'Adventures in You...",Secret City Records,[Secret City Records],True,[Secret City Records]


In [ ]:
i = 1
example = df.iloc[i].to_dict()
example['question']

"In what part of Florida is Tom Denney's birthplace located?"

In [ ]:
from bellek.qa.llm import make_question_answer_func

qa_func = make_question_answer_func()
retrieval_func = lambda docs, query: docs

In [ ]:
from bellek.utils import jprint

pipeline = BaselineSingleHop(qa_func, retrieval_func)
output = pipeline(example).dict()
print("Question:", example['question'])
print("Reference answer:", example['answer'])
print("Predicted answer:", output['answer'])
print("Reasoning:", output['reasoning'])
jprint(json.loads(output['raw_output']))

Question: In what part of Florida is Tom Denney's birthplace located?
Reference answer: in Northern Florida
Predicted answer: Northern Florida.
Reasoning: Tom Denney was mentioned to be from Ocala, Florida in the context information.
{
  "reasoning": "Tom Denney was mentioned to be from Ocala, Florida in the context information.",
  "answer": "Northern Florida."
}


In [ ]:
from bellek.utils import jprint

pipeline = BaselineMultiHop(qa_func, retrieval_func)
output = pipeline(example).dict()
print("Question:", example['question'])
print("Reference answer:", example['answer'])
print("Predicted answer:", output['answer'])
print("Reasoning:", output['reasoning'])
jprint(json.loads(output['raw_output']))


Question: In what part of Florida is Tom Denney's birthplace located?
Reference answer: in Northern Florida
Predicted answer: Northern Florida
Reasoning: Ocala is located in Northern Florida.
[
  {
    "question": "Tom Denney >> place of birth",
    "query": "Tom Denney >> place of birth",
    "context": "# Ocala, Florida\nOcala (/ o\u028a \u02c8k\u00e6l\u0259 / oh - KAL - \u0259) is a city located in Northern Florida. As of the 2013 census, its population, estimated by the United States Census Bureau, was 57,468, making it the 45th most populated city in Florida.\n# Jasmine Estates, Florida\nJasmine Estates is a census-designated place (CDP) in Pasco County, Florida, United States. The population was 18,989 at the 2010 census.\n# Ridgecrest, Florida\nRidgecrest is a census-designated place (CDP) in Pinellas County, Florida, United States. The population was 2,558 at the 2010 census.\n# Villas, Florida\nVillas is a census designated place (CDP) in Lee County, Florida, United States. Th

In [ ]:
mini_df = df.sample(3)

In [ ]:
shqa_df, scores = benchmark(mini_df, BaselineSingleHop(qa_func, retrieval_func))
print(scores)
shqa_df

  0%|          | 0/3 [00:00<?, ?it/s]

{'exact_match': 0.3333333333333333, 'f1': 0.3333333333333333, 'fuzzy_match': 0.3333333333333333}


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers,predicted_answer,raw_llm_output,exact_match,fuzzy_match
33,2hop__333020_69048,"[{'idx': 0, 'title': 'Supreme Court of India',...",Who is the Chief Judge of the place where Orez...,"[{'id': 333020, 'question': 'Orezi >> place of...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith],Adama Lamikanra,reasoning='Identify the birthplace of Orezi. F...,False,False
34,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River],Chao Phraya River,reasoning='The Pa Sak River flows into the Cha...,True,True
78,2hop__477492_240386,"[{'idx': 0, 'title': 'Necessary Records', 'par...",What is the record label of the Metalworks Stu...,"[{'id': 477492, 'question': 'Metalworks Studio...",TML Entertainment,[TML Entertainment],True,[TML Entertainment],Metalworks Studios,reasoning='Metalworks Studio founder is Gil Mo...,False,False


In [ ]:
mhqa_df, scores = benchmark(mini_df, BaselineMultiHop(qa_func, retrieval_func))
print(scores)
mhqa_df

  0%|          | 0/3 [00:00<?, ?it/s]

{'exact_match': 0.3333333333333333, 'f1': 0.3333333333333333, 'fuzzy_match': 0.3333333333333333}


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers,predicted_answer,raw_llm_output,exact_match,fuzzy_match
33,2hop__333020_69048,"[{'idx': 0, 'title': 'Supreme Court of India',...",Who is the Chief Judge of the place where Orez...,"[{'id': 333020, 'question': 'Orezi >> place of...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith],Honorable Justice Abiodun Smith,reasoning='Identify the entity mentioned to be...,True,True
34,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River],Confluence point,reasoning='A watercourse typically flows into ...,False,False
78,2hop__477492_240386,"[{'idx': 0, 'title': 'Necessary Records', 'par...",What is the record label of the Metalworks Stu...,"[{'id': 477492, 'question': 'Metalworks Studio...",TML Entertainment,[TML Entertainment],True,[TML Entertainment],unknown,reasoning='Gil Moore was one of the founders o...,False,False


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()